In [3]:
import sys
import math
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

file_train = open("/content/drive/MyDrive/Colab Notebooks/COL774/Assignment4/2018CS50098/train.csv", "r")
data_train = torch.Tensor([[float(cell) for cell in row.split(",")] for row in file_train])
file_test1 = open("/content/drive/MyDrive/Colab Notebooks/COL774/Assignment4/2018CS50098/public_test.csv", "r")
data_test1 = torch.Tensor([[float(cell) for cell in row.split(",")] for row in file_test1])
file_test2 = open("/content/drive/MyDrive/Colab Notebooks/COL774/Assignment4/2018CS50098/private.csv", "r")
data_test2 = torch.Tensor([[float(cell) for cell in row.split(",")] for row in file_test2])

class CNN(nn.Module):

    def __init__(self, n, batch_size, r):
        super(CNN, self).__init__()
        self.root_n = int(math.sqrt(n))
        self.conv = nn.Sequential(
            nn.Conv2d(1, 64, (3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, (3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, (3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d((2, 2), stride=2, padding=0),
            nn.Dropout(p=0.5),
            nn.Conv2d(64, 128, (3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, (3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, (3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d((2, 2), stride=2, padding=0),
            nn.Dropout(p=0.5),
            nn.Conv2d(128, 256, (3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, (3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, (3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d((2, 2), stride=2, padding=0),
            nn.Dropout(p=0.5),
            nn.Conv2d(256, 512, (3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, (3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, (3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d((2, 2), stride=2, padding=0),
            nn.Dropout(p=0.5)
        )
        self.linear = nn.Sequential(
            nn.Linear(512*3*3,512),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(64,r)
        )
        self.batch_size = batch_size
        self.optimizer = torch.optim.SGD(self.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-5)
        self.loss_function = nn.CrossEntropyLoss()

    def forward(self, X):
        X = X.view(-1, 1, self.root_n, self.root_n)
        X = self.conv(X)
        X = X.view(-1, 3*3*512)
        X = self.linear(X)
        return X

    def train(self, data):
        if torch.cuda.is_available():
            self = self.cuda()
            data = data.cuda()
        max_epoch = 200
        for epoch in range(max_epoch):
            cost = 0
            data = data[torch.randperm(data.shape[0]),:]
            for batch in range(math.ceil(data.shape[0]/self.batch_size)):
                self.optimizer.zero_grad()
                output = self(data[batch*self.batch_size:(batch+1)*self.batch_size,1:])
                loss = self.loss_function(output, data[batch*self.batch_size:(batch+1)*self.batch_size,0].long())
                loss.backward()
                self.optimizer.step()
                cost += float(loss.data) * data[batch*self.batch_size:(batch+1)*self.batch_size,:].shape[0]
            print(f"Epochs = {epoch+1} and Loss = {round(cost/data.shape[0],6)}")
        if torch.cuda.is_available():
            self = self.cpu()
            data = data.cpu()

    def test(self, data):
        if torch.cuda.is_available():
            self = self.cuda()
            data = data.cuda()
        cost = 0
        correct = 0
        for batch in range (math.ceil(data.shape[0]/self.batch_size)):
            output = self(data[batch*self.batch_size:(batch+1)*self.batch_size,1:])
            loss = self.loss_function(output, data[batch*self.batch_size:(batch+1)*self.batch_size,0].long())
            cost += float(loss.data) * data[batch*self.batch_size:(batch+1)*self.batch_size,:].shape[0]
            prediction = F.softmax(output.data, dim=1).max(1)[1]
            correct += float(prediction.eq(data[batch*self.batch_size:(batch+1)*self.batch_size,0]).sum())
        print(f"Accuracy = {round(100*correct/data.shape[0],2)}% and Loss = {round(cost/data.shape[0],6)}")
        if torch.cuda.is_available():
            self = self.cpu()
            data = data.cpu()

    def predict(self, data):
        if torch.cuda.is_available():
            self = self.cuda()
            data = data.cuda()
        y = []
        for batch in range (math.ceil(data.shape[0]/self.batch_size)):
            output = self(data[batch*self.batch_size:(batch+1)*self.batch_size,1:])
            prediction = F.softmax(output.data, dim=1).max(1)[1]
            y.extend(prediction)
        y = [f"Id,Prediction"] + [f"{i+1},{y[i]}" for i in range(len(y))]
        if torch.cuda.is_available():
            self = self.cpu()
            data = data.cpu()
        return y

cnn = CNN(data_train.shape[1]-1, 64, 7)
cnn.train(data_train) 
cnn.test(data_train)
cnn.test(data_test1)
np.savetxt("/content/drive/MyDrive/Colab Notebooks/COL774/Assignment4/2018CS50098/y_c.csv", cnn.predict(data_test2), fmt="%s", delimiter="\n")

Epochs = 1 and Loss = 1.825229
Epochs = 2 and Loss = 1.792486
Epochs = 3 and Loss = 1.75542
Epochs = 4 and Loss = 1.644001
Epochs = 5 and Loss = 1.535361
Epochs = 6 and Loss = 1.471291
Epochs = 7 and Loss = 1.419985
Epochs = 8 and Loss = 1.39299
Epochs = 9 and Loss = 1.368972
Epochs = 10 and Loss = 1.345035
Epochs = 11 and Loss = 1.314111
Epochs = 12 and Loss = 1.303246
Epochs = 13 and Loss = 1.271466
Epochs = 14 and Loss = 1.255286
Epochs = 15 and Loss = 1.235623
Epochs = 16 and Loss = 1.221437
Epochs = 17 and Loss = 1.204054
Epochs = 18 and Loss = 1.18809
Epochs = 19 and Loss = 1.181746
Epochs = 20 and Loss = 1.155309
Epochs = 21 and Loss = 1.138495
Epochs = 22 and Loss = 1.120703
Epochs = 23 and Loss = 1.121047
Epochs = 24 and Loss = 1.105348
Epochs = 25 and Loss = 1.088078
Epochs = 26 and Loss = 1.077777
Epochs = 27 and Loss = 1.060622
Epochs = 28 and Loss = 1.058017
Epochs = 29 and Loss = 1.039068
Epochs = 30 and Loss = 1.028075
Epochs = 31 and Loss = 1.004852
Epochs = 32 and Loss